<a href="https://colab.research.google.com/github/HSE-LAMBDA/MLDM-2022/blob/master/04-regularization/Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boston housing dataset

In this example we'll try to predict housing prices.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [ ]:
data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

Information about the dataset:

In [ ]:
print(data.DESCR)

# Exploring the data

Let's just see how the target depends on individual features.

In [ ]:
plt.figure(figsize=(15, 15))

grid_size = int(np.ceil(X.shape[1]**0.5))

for i, (name, x) in enumerate(X.iteritems(), 1):
  plt.subplot(grid_size, grid_size, i)
  plt.scatter(x, data.target)
  plt.xlabel(name)

Let's start by trying a simple linear regression model on the features with the most obvious correlation with the target. We'll also scale the features manually.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [ ]:
columns = ["CRIM", "RM", "LSTAT"]

X_subset = X[columns]
X_subset /= X_subset.max()

In [ ]:
model = make_pipeline(
    PolynomialFeatures(9, include_bias=False), # Can you calculate how many features this will result in? :)
    LinearRegression()
)

model.fit(X_subset, y)
print('mse = ', mean_squared_error(y, model.predict(X_subset)))

plt.figure(figsize=(15, 4))
for i, c in enumerate(columns, 1):
  plt.subplot(1, len(columns), i)
  plt.scatter(X[c], y, label='data')
  plt.scatter(X[c], model.predict(X_subset), label='prediction')
  plt.legend()

# Splitting the data to train and validation parts

Looks like the fit from above is reasonable, right?

In fact, we cannot know this yet: we fitted and predicted on the same data. Let's split our dataset to get a more reasonable estimate of the prediction error.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=50, random_state=39)

In [ ]:
model = make_pipeline(
    PolynomialFeatures(9, include_bias=False),
    LinearRegression()
)

model.fit(X_train, y_train)

print('train mse = ', mean_squared_error(y_train, model.predict(X_train)))
print('test mse = ', mean_squared_error(y_test, model.predict(X_test)))

That's quite an error we have on the test set!

Let's look at the prediction.

In [ ]:
plt.figure(figsize=(15, 4))
for i, c in enumerate(columns, 1):
  plt.subplot(1, len(columns), i)
  plt.scatter(X_test[c], y_test, label='data')
  plt.scatter(X_test[c], model.predict(X_test), label='prediction')
  plt.legend()

That's because our parameter values at the solution are enormous:

In [ ]:
model.named_steps['linearregression'].coef_

# L2 regularization (ridge regression)

Let's regularize the solution!

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
model = make_pipeline(
    PolynomialFeatures(9, include_bias=False),
    Ridge(alpha=1.)
)

model.fit(X_train, y_train)

print('train mse = ', mean_squared_error(y_train, model.predict(X_train)))
print('test mse = ', mean_squared_error(y_test, model.predict(X_test)))

In [ ]:
plt.figure(figsize=(15, 4))
for i, c in enumerate(columns, 1):
  plt.subplot(1, len(columns), i)
  plt.scatter(X_test[c], y_test, label='data')
  plt.scatter(X_test[c], model.predict(X_test), label='prediction')
  plt.legend()

In [ ]:
model.named_steps['ridge'].coef_

Now we'll study how losses and parameter values depend on the regularization power.

In [ ]:
from tqdm import tqdm

In [ ]:
reg_powers = np.logspace(-12, 5, 18 * 5, base=10)


train_mse = []
test_mse = []

params = []

for alpha in tqdm(reg_powers):
  linear_model = Ridge(alpha=alpha)
  model = make_pipeline(
    PolynomialFeatures(9, include_bias=False),
    linear_model
  )
  model.fit(X_train, y_train)

  params.append(
      np.append(linear_model.coef_,
                linear_model.intercept_)
  )

  train_mse.append(mean_squared_error(y_train, model.predict(X_train)))
  test_mse.append(mean_squared_error(y_test, model.predict(X_test)))

params = np.array(params)


plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)

plt.plot(1. / reg_powers, train_mse, label='train')
plt.plot(1. / reg_powers, test_mse, label='test')
plt.ylabel('MSE')
plt.xlabel('inverse regularization power')
plt.legend()
plt.xscale('log')
plt.yscale('log')

plt.subplot(1, 2, 2)
plt.plot(1. / reg_powers, np.abs(params));
plt.xlabel("inverse regularization power")
plt.ylabel("parameter magnitude")
plt.xscale('log')
plt.yscale('log')

# L1 regularization (lasso regression)

Here's a similar study with the Lasso regression:

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
reg_powers = np.logspace(-4, 1, 6 * 5, base=10)

train_mse = []
test_mse = []

params = []

for alpha in tqdm(reg_powers):
  # Lasso doesn't have an analytic solution. Instead it
  # utilizes an iterative procedure, which for small
  # alpha values may take a while to converge.
  linear_model = Lasso(alpha=alpha, max_iter=1000000)
  model = make_pipeline(
    PolynomialFeatures(9, include_bias=False),
    linear_model
  )
  model.fit(X_train, y_train)

  params.append(
      np.append(linear_model.coef_,
                linear_model.intercept_)
  )

  train_mse.append(mean_squared_error(y_train, model.predict(X_train)))
  test_mse.append(mean_squared_error(y_test, model.predict(X_test)))

params = np.array(params)

plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)

plt.plot(1. / reg_powers, train_mse, label='train')
plt.plot(1. / reg_powers, test_mse, label='test')
plt.ylabel('MSE')
plt.xlabel('inverse regularization power')
plt.legend()
plt.xscale('log')

plt.subplot(1, 3, 2)
plt.plot(1. / reg_powers, np.abs(params));
plt.xlabel("inverse regularization power")
plt.ylabel("parameter magnitude")
plt.xscale('log')
plt.yscale('log')

plt.subplot(1, 3, 3)
plt.plot(1. / reg_powers, np.isclose(params, 0.).sum(axis=1));
plt.xlabel("inverse regularization power")
plt.ylabel("# zero parameters")
plt.xscale('log')

# Bonus. What features are the most powerful?

Let's see what features are most powerful for a reasonably performing model (e.g. 1/alpha = 100) **(2 points)**:

In [ ]:
model = make_pipeline(
  PolynomialFeatures(9, include_bias=False),
  Lasso(alpha=0.01, max_iter=1000000)
)
model.fit(X_train, y_train)

print('train mse = ', mean_squared_error(y_train, model.predict(X_train)))
print('test mse = ', mean_squared_error(y_test, model.predict(X_test)))

Some hints:
 - You can explore the feature names using `get_feature_names` method of the `PolynomialFeatures` class (plug the list of original feature names to get reasonable output).
 - `model.named_steps['polynomialfeatures']` to get the `PolynomialFeatures` preprocessor of our model.
 - `model.named_steps['lasso'].coef_` to get the parameters of the linear model
 - `np.argwhere` to find indices of non-zero elements of an array


In [ ]:
<YOUR CODE>